In [9]:
from house_project.Utility.common import *
from house_project.Config.config_entity import DataTransfomationConfig


In [10]:
class DataTransfomation:
    def __init__(self):
        self.data_transform = DataTransfomationConfig()
        self.schema = Read_Yaml(self.data_transform.schema_path)
        
        

In [11]:
schema = Read_Yaml('D:\End-to-End-SL-House-Price-Prediction\schema.yaml')

[ 2025-02-15 01:31:14,930 ] 18 root - INFO - Read the D:\End-to-End-SL-House-Price-Prediction\schema.yaml Yaml file


In [12]:
drop_columns = schema.get('drop_columns',[])
#if isinstance(drop_columns,str):
 #   drop_columns =drop_columns.split()

In [13]:
drop_columns

['Title',
 'Sub_title',
 'Address',
 'Description',
 'Post_URL',
 'Seller_name',
 'Seller_type',
 'published_date',
 'Geo_Address',
 'Lat',
 'Lon']

In [14]:
import pandas as pd 

In [15]:
data = pd.read_csv('house_prices.csv')
data.head(2)

,Title,Sub_title,Price,Address,Baths,Land size,Beds,House size,Location,Description,Post_URL,Seller_name,Seller_type,published_date,Geo_Address,Lat,Lon
0,House with Land for Sale in Matara for sale,"Posted on 06 Nov 2:32 pm, Matara City, Matara","Rs 5,400,000","Gangodagama Roard,Hakmana,Matara.",1,50.0 perches,3,"1,600.0 sqft","Matara City, Matara","Land for sale with house Matara,Hakmana Gangod...",https://ikman.lk/en/ad/house-with-land-for-sal...,Ishara Dilshan,Member,2021-11-06 14:32:00,"Matara City, Matara, Sri Lanka",80.500000,6.166670
1,à¶ à¶½à·à¶­à· à¶à·à¶¸à¶» 3 à¶ à¶à·à· à...,"Posted on 24 Oct 7:27 am, Athurugiriya, Colombo","Rs 16,800,000",Athurugiriya Galwarusapare,3,8.0 perches,3,"1,480.0 sqft","Athurugiriya, Colombo",*House For Sale In Athurugiriya *Galwarusapare...,https://ikman.lk/en/ad/alut-kaamr-3-k-grh-bhaa...,DILRUWAN REAL ESTATE,Premium-Member,2021-10-24 07:27:00,"Athurugiriya, Colombo, Sri Lanka",79.989929,6.877246


In [16]:
data.drop(columns=drop_columns,axis=1)

,Price,Baths,Land size,Beds,House size,Location
0,"Rs 5,400,000",1,50.0 perches,3,"1,600.0 sqft","Matara City, Matara"
1,"Rs 16,800,000",3,8.0 perches,3,"1,480.0 sqft","Athurugiriya, Colombo"
2,"Rs 20,000,000",2,20.0 perches,3,"2,800.0 sqft","Kelaniya, Gampaha"
3,"Rs 187,000,000",5,22.0 perches,5,"4,000.0 sqft","Colombo 6, Colombo"
4,"Rs 1,300,000",1,6.3 perches,2,900.0 sqft,"Dehiwala, Colombo"
...,...,...,...,...,...,...
15322,"Rs 6,750,000",1,15.0 perches,3,"1,200.0 sqft","Nittambuwa, Gampaha"
15323,"Rs 70,000,000",2,7.0 perches,3,"1,100.0 sqft","Dehiwala, Colombo"
15324,"Rs 44,000,000",4,20.0 perches,5,"2,750.0 sqft","Maharagama, Colombo"
15325,"Rs 15,000,000",2,5.0 perches,2,"1,350.0 sqft","Dehiwala, Colombo"


In [17]:
import numpy as np

In [ ]:
def remove_outliers_iqr(data,columns,multiplier=1.5):
    if isinstance(columns, str):
        columns = [columns]
    for col in columns:
        if col in data.columns:
            Q1 = np.percentile(data[col],25,method='midpoint')
            Q3 = np.percentile(data[col],75,method='midpoint')
            IQR = Q3-Q1
            lower_bound = Q1 - (multiplier * IQR)
            upper_bound = Q3 + (multiplier * IQR)
                
            data = data[(data[col] > lower_bound) & (data[col] < upper_bound)] 
    return data           
                

In [4]:
from house_project.Config.config_entity import DataTransfomationConfig
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np 
from house_project import logging
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from house_project.Utility.common import *
from house_project.Utility.feature_engineering import FeatureEngineering
import os 


class DataTransfomation:
    def __init__(self):
        self.data_transform = DataTransfomationConfig()
        self.schema = Read_Yaml(self.data_transform.schema_path)
        self.data_clean = FeatureEngineering()
        csv_file = check_csv_occur(self.data_transform.ingest_data_root)
        data = pd.read_csv(csv_file)
        data.head()